# <font color='black'>Data Sciente Academy</font>
## <font color='black'>Generative AI and LLMs for Natural Language Processing</font>
## <font color='black'>Study Case 1 - LLM for text generation - GPT-2-Large written in Python</font>

---

## Instalando e Carregando Pacotes

Watermark allows to create a "watermark" with the current date and time, the version of Python, and the version of the Jupyter notebook. It is useful for keeping track of the version of the notebook and the environment in which it was run. This is useful for reproducibility and for sharing the notebook with others.
*   -q is for quiet mode, which suppresses output messages.
*   -U is for upgrade, which upgrades the package to the latest version.

In [6]:
%pip install -q -U watermark

Tensorflow is a library for machine learning and deep learning. It contains functions and classes for building and training neural networks including, convolutional neural networks (CNNs), recurrent neural networks (RNNs) and
transformer architecture.

In [7]:
%pip install -q tensorflow==2.19.0

A biblioteca Transformers do Python, criada pela Hugging Face, fornece ferramentas para uso simplificado de modelos de inteligência artificial baseados em transformers. É usada principalmente em NLP para tarefas como geração de texto, classificação, tradução e análise de sentimentos. Permite fácil integração de modelos pré-treinados, acelerando o desenvolvimento de soluções avançadas.

In [8]:
%pip install -q -U transformers==4.50.3

In [ ]:
#%pip install -q -U torch

Note: you may need to restart the kernel to use updated packages.


https://huggingface.co/

In [9]:
# Imports
import tensorflow as tf
#import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [10]:
%reload_ext watermark
%watermark -a "Lucas P. Lira"

Author: Lucas P. Lira



<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

## Funções Usadas Para Carregar Um LLM Pré-Treinado

**GPT2LMHeadModel**: Esta é a classe modelo para o GPT-2 (Generative Pretrained Transformer 2), um modelo de linguagem poderoso que pode gerar texto semelhante ao humano. A parte 'LMHead' indica que esta variante do modelo inclui um cabeçote de modelagem de linguagem no topo do modelo GPT-2 básico, o que o torna adequado para tarefas como geração de texto.

**GPT2Tokenizer**: Este é um tokenizer projetado especificamente para o modelo GPT-2. Um tokenizer é usado para converter texto em um formato que possa ser compreendido e processado pelo modelo. Para GPT-2, isso envolve a conversão de texto em tokens (como palavras ou subpalavras) e a codificação desses tokens como valores numéricos.

## Carregando o Tokenizador

https://huggingface.co/gpt2-large

In [29]:
# Carrega o tokenizador
# device_map="auto" permite que o modelo utilize automaticamente a GPU disponível, se houver uma.
# Caso contrário, ele usará a CPU.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large", device_map="auto")

In [30]:
# Define o conjunto de regras de como os dados serão processados e retorna em um formato em que o modelo é capaz de processar.
tokenizer

GPT2Tokenizer(name_or_path='gpt2-large', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

**from_pretrained**: Este é um método da classe GPT2Tokenizer. O método from_pretrained é usado para carregar um tokenizer que já foi treinado em um conjunto de dados específico. Neste caso, está treinado para trabalhar com o modelo GPT-2.

**"gpt2-large"**: Este é um argumento de string para o método from_pretrained. Ele especifica qual variante do tokenizer do modelo GPT-2 você deseja usar. O modelo gpt2-large é uma versão maior do modelo GPT-2, o que significa que possui mais parâmetros e pode potencialmente gerar texto mais coerente e contextualmente relevante em comparação com versões menores. O tokenizer para gpt2-large é treinado especificamente para funcionar bem com esta variante de modelo.

## Carregando o Modelo Pré-Treinado

In [31]:
# Carrega o modelo
model = GPT2LMHeadModel.from_pretrained("gpt2-large", device_map="auto", pad_token_id = tokenizer.eos_token_id)

In [32]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3840, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=1280)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=5120, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=5120)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

**pad_token_id**: Este parâmetro define o token que o modelo usa para preenchimento. O preenchimento é usado para preencher sequências mais curtas para corresponder ao comprimento da sequência mais longa ao processar um lote de dados de texto.

**tokenizer.eos_token_id**: O eos_token_id é o ID do token de 'fim da string' do tokenizer. Esta linha define o ID do token de preenchimento como igual ao ID do token de final de string. Ele informa ao modelo que o preenchimento deve ser tratado de forma semelhante ao token de final de string, o que é importante para manter a consistência na forma como o modelo processa sequências de texto.

## Gerando Texto com o LLM

In [34]:
# Texto inicial
prompt = 'What is Artificial Intelligence'

In [35]:
# Tokeniza o texto do prompt
input_ids = tokenizer.encode(prompt, return_tensors = 'pt')

**encode**: Este é um método do tokenizer. Ele recebe uma sequência de texto e a converte em uma lista de tokens numéricos. Cada token corresponde a uma palavra ou parte de uma palavra.

**return_tensors='pt'**: Este argumento diz ao tokenizer para retornar os tokens em um formato adequado para PyTorch (indicado por 'pt').

In [36]:
# Visualiza
input_ids

tensor([[ 2061,   318, 35941,  9345]])

In [40]:
input_ids = input_ids.to('cuda')

O tensor de saída representa um tensor PyTorch contendo uma sequência de tokens numéricos. Esses tokens são o resultado do processamento de texto por meio de um tokenizer, especificamente o tokenizer GPT-2 nesse caso.

O tokenizer possui um vocabulário de tokens (palavras, partes de palavras, símbolos, etc.) e cada pedaço de texto exclusivo recebe um número específico. Por exemplo, a palavra “olá” pode ser transformada no número 1256, “mundo” pode se tornar 5678 e assim por diante.

A sequência acima corresponde à forma tokenizada da frase de entrada. Cada número é mapeado para uma palavra ou parte de uma palavra nessa frase.

Os colchetes duplos indicam que este é um tensor bidimensional. Neste contexto uma única sequência (uma frase) está sendo representada, portanto, apenas uma linha neste tensor 2D.

Os valores numéricos não têm significado inerente sem o contexto do vocabulário do tokenizer. Para entender qual texto cada número representa, você precisaria procurar esses IDs no vocabulário do tokenizer.

Esse formato de tensor é o que você normalmente alimenta em um modelo de aprendizado de máquina como o GPT-2 para executar tarefas como geração de texto, classificação ou resposta a perguntas. O modelo lê esses números e usa sua rede neural treinada para interpretá-los e gerar resultados apropriados com base em seu treinamento.

In [41]:
# Gerando o texto a partir do prompt com o texto inicial
texto_gerado = model.generate(input_ids,
                              max_length = 100,
                              num_beams = 5,
                              no_repeat_ngram_size = 2,
                              early_stopping = True)

**max_length=100**: Este parâmetro define o comprimento máximo da sequência a ser gerada. O valor 100 inclui o comprimento do texto de entrada (o contexto) e o novo texto que o modelo irá gerar. O modelo para de gerar tokens adicionais quando esse comprimento é atingido.

**num_beams=5**: Este parâmetro permite a pesquisa de feixe com 5 feixes. A pesquisa em feixe é uma técnica usada em PLN para geração de texto onde o modelo considera várias próximas palavras possíveis em cada etapa e mantém as sequências (ou "feixes") mais promissoras de tokens em cada etapa. Usar 5 feixes significa que o modelo rastreia 5 sequências potenciais em cada etapa, o que pode levar a resultados de maior qualidade, mas requer mais recursos computacionais.

**no_repeat_ngram_size=2**: Esta configuração evita que o modelo repita os mesmos n-gramas (neste caso, sequências de 2 tokens) no texto de saída. Ajuda a reduzir a repetitividade no texto gerado.

**early_stopping=True**: Este parâmetro informa ao modelo para parar de gerar texto assim que todos os candidatos ao feixe atingirem o token de final da frase. Pode tornar o processo de geração mais eficiente, interrompendo a busca assim que uma saída satisfatória for encontrada.

In [42]:
texto_gerado

tensor([[ 2061,   318, 35941,  9345,    30,   198,   198,  8001,  9542,  4430,
           357, 20185,     8,   318,   257,  8478,   286,  3644,  3783,   326,
          7529,   351,  3644,  4056,   326,   389,  1498,   284,  2193,   290,
          6068,   284,   511,  2858,    13,  9552,   468,   587,  1088,   329,
           257,   890,   640,    11,   475,   340,   318,   691,  2904,   326,
          9061,   423,  1716,  6007,   286,  9489,  8861,   326,   547,  1752,
          1807,   284,   307,  3675,   262,  9889,   286,  1692,  9791,    13,
          1114,  1672,    11,   257,  3644,  1430,   460,  2193,   703,   284,
           711,   257,  2008,   983,   416,  2712,   262,   976,   983,   625,
           290,   625,   757,    13,   770,   318,  1444, 37414,  4673,    13]],
       device='cuda:0')

Texto? Como assim?

Isso é o que o modelo gera realmente. Vamos agora converter isso em algo que faça sentido.

## Decodificando o Resultado com o Tokenizador

In [43]:
print(tokenizer.decode(texto_gerado[0], skip_special_tokens = True))

What is Artificial Intelligence?

Artificial intelligence (AI) is a branch of computer science that deals with computer programs that are able to learn and adapt to their environment. AI has been around for a long time, but it is only recently that computers have become capable of performing tasks that were once thought to be beyond the capabilities of human beings. For example, a computer program can learn how to play a video game by playing the same game over and over again. This is called reinforcement learning.


O tokenizer não apenas converte texto em tokens (representações numéricas), mas também pode fazer o inverso – convertendo tokens de volta em texto.

**skip_special_tokens=True**: Este argumento diz ao decodificador para ignorar tokens especiais, como tokens de preenchimento ou tokens de fim de sequência que são usados para processamento do modelo, mas não são significativos no texto final gerado.

In [44]:
%watermark -a "Lucas P. Lira" -v -p tensorflow,torch,transformers

Author: Lucas P. Lira

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 7.34.0

tensorflow  : 2.19.0
torch       : 2.6.0+cu124
transformers: 4.50.3



In [45]:
%watermark -v -m

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [46]:
%watermark --iversions

transformers: 4.50.3
tensorflow  : 2.19.0



# Fim